In [1]:
# ToDo: change core library for compat v1
import tensorflow as tf

In [2]:
import json

In [3]:
from google.cloud import storage

In [4]:
storage_client = storage.Client()

In [5]:
_BUCKET_NAME = 'cbidmltsf'

In [6]:
bucket = storage_client.get_bucket(_BUCKET_NAME)

In [7]:
model_dir = '007_CPU'

In [8]:
# models path has changed from dplstm to models
saved_model_path = 'models/{0}/export/exporter'.format(model_dir)
saved_model_path

'models/007_CPU/export/exporter'

In [9]:
# https://hackersandslackers.com/manage-files-in-google-cloud-storage-with-python/

In [10]:
def list_files(bucketFolder):
    """List all files in GCP bucket."""
    files = bucket.list_blobs(prefix=bucketFolder)
    fileList = [file.name for file in files if '.' in file.name]
    return fileList

In [11]:
all_files = list_files(saved_model_path)

In [12]:
[file for file in all_files]

['models/007_CPU/export/exporter/1594219862/saved_model.pb',
 'models/007_CPU/export/exporter/1594219862/variables/variables.data-00000-of-00001',
 'models/007_CPU/export/exporter/1594219862/variables/variables.index']

In [13]:
# isolate the names of the subdirectories in export/exporter (one for each training process)
prefix_length = len(saved_model_path)

In [14]:
latest_saved_model_id = sorted(list(set([file[prefix_length+1:prefix_length+11] for file in all_files])))[-1]

In [15]:
_LATEST_SAVED_MODEL_DIR = 'gs://{0}/{1}/{2}'.format(_BUCKET_NAME,
                                                    saved_model_path,
                                                    latest_saved_model_id)
_LATEST_SAVED_MODEL_DIR

'gs://cbidmltsf/models/007_CPU/export/exporter/1594219862'

In [16]:
# build a prediction function
predict_fn = tf.contrib.predictor.from_saved_model(_LATEST_SAVED_MODEL_DIR)
# following the warning that results from executing the previous line
# predict_fn = tf.compat.v1.saved_model.load(_LATEST_SAVED_MODEL_DIR)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://cbidmltsf/models/007_CPU/export/exporter/1594219862/variables/variables


In [17]:
_SLDB_ID = 'CPE04015_desbI_H_2017-04-01_00:00:00_2018-02-28_23:00:00_H008001001_D007024001_W004168001'

In [18]:
# un-comment the following line to predict on test set
# test_dataset_filename = 'gs://cbidmltsf/sldbs/{0}/test.tfrecord'.format(_SLDB_ID)
# use evaluation dataset as testing dataset
test_dataset_filename = 'gs://cbidmltsf/sldbs/{0}/test.tfrecord'.format(_SLDB_ID)

In [19]:
test_dataset_filename

'gs://cbidmltsf/sldbs/CPE04015_desbI_H_2017-04-01_00:00:00_2018-02-28_23:00:00_H008001001_D007024001_W004168001/test.tfrecord'

In [20]:
test_dataset = tf.data.TFRecordDataset(test_dataset_filename)

In [21]:
test_dataset

<TFRecordDatasetV1 shapes: (), types: tf.string>

In [25]:
dataset = test_dataset.map(lambda row: row)
# ToDo: find out if there is a faster way to build the predictions list,
#   with no iterator on the dataset
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [23]:
sess = tf.InteractiveSession()

In [24]:
# ToDo: load a single serialized example to test the prediction funcion on
value = sess.run(next_element)
print(value)

b'\n\xcf\x02\nm\n\x05oh_dh\x12d\x12b\n`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\n\x12\n\x06target\x12\x08\x12\x06\n\x04\xce\xfb\xc5>\n.\n\x06hourly\x12$\x12"\n \xa7r\xb7>\xfck\xdd>\xc3\xb8\xe0>(\xa4\x03?_\xfe\x1b?\x04\xab#?QF\xf7>9\r\x06?\n\x1e\n\x06weekly\x12\x14\x12\x12\n\x10\xb6\xfe\xcd>\xa1\xa9\x19?-y\x9e>g\x8e\xa9>\n)\n\x05oh_wd\x12 \x12\x1e\n\x1c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80?\x00\x00\x00\x00\x00\x00\x00\x00\n)\n\x05daily\x12 \x12\x1e\n\x1cg\x8e\xa9>\x7f\xd4\xc7>\x10\xb3\x14?J\xfc\x0c?\x81\'\x1b?)\xbd\x17?\xc8\x81\x0e?\n$\n\ttimestamp\x12\x17\n\x15\n\x132018-02-23 14:00:00'


In [26]:
predictions_list = []
with tf.compat.v1.Session() as sess:
    try:
        while True:
            example = sess.run(next_element)
            predictions_list.append(predict_fn({'example_bytes': example}))
    except:
        pass

In [27]:
len(predictions_list)

0

In [68]:
_SLDB_ID

'CPE04015_desbI_H_2017-04-01_00:00:00_2018-02-28_23:00:00_H008001001_D007024001_W004168001'

In [73]:
offset = 58
_SLDB_ID[offset:offset+3]

'008'

In [ ]:
def upload_files(bucketName):
    """Upload files to GCP bucket."""
    files = [f for f in listdir(localFolder) if isfile(join(localFolder, f))]
    for file in files:
        localFile = localFolder + file
        blob = bucket.blob(bucketFolder + file)
        blob.upload_from_filename(localFile)
    return f'Uploaded {files} to "{bucketName}" bucket.'

In [75]:
# upload a file to the model_dir in GS

In [76]:
storage_client

In [77]:
bucket

<Bucket: cbidmltsf>

In [79]:
bucket

<Bucket: cbidmltsf>

In [88]:
local_file = 'predictions.html'

In [96]:
plot_blob_id = 'dplstm/{0}/{1}'.format(model_dir, local_file)
plot_blob_id

'dplstm/086403_073202_043201_CPU_07_00/predictions.html'

In [97]:
plot_blob = bucket.blob(plot_blob_id)

In [98]:
plot_blob.upload_from_filename(local_file)

In [155]:
with open('sldb.json', 'r') as filename:
    sldb = json.load(filename)

In [156]:
objective_shapes = {
    'hourly': [sldb['components']['hourly']['m'], 1],
    'daily': [sldb['components']['daily']['m'], 1],
    'weekly': [sldb['components']['weekly']['m'], 1],
    'target': [sldb['components']['hourly']['no_targets'], 1],
    'oh_wd': [7, 1],  # Monday to Sunday
    'oh_dh': [24, 1],  # midnight to 23:00
    'timestamp': [sldb['components']['hourly']['no_targets'], 1]
}

In [157]:
read_features = {
    'hourly': tf.io.VarLenFeature(dtype=tf.float32),
    'daily': tf.io.VarLenFeature(dtype=tf.float32),
    'weekly': tf.io.VarLenFeature(dtype=tf.float32),
    'target': tf.io.VarLenFeature(dtype=tf.float32),
    'oh_wd': tf.io.VarLenFeature(dtype=tf.float32),
    'oh_dh': tf.io.VarLenFeature(dtype=tf.float32),
    'timestamp': tf.io.VarLenFeature(dtype=tf.string)
}

In [158]:
# do not pass objective shapes as a parameter, get them from an outer scope variable instead
def _parse_dataset_function(example_proto):
    # parse the input tf.Example proto using the dictionary above
    row = tf.io.parse_single_example(example_proto, read_features)
    # pass objective shape as a list of lists [hourly_shape, daily_shape, weekly_shape]
    hourly = tf.reshape(row['hourly'].values, objective_shapes['hourly'])
    daily = tf.reshape(row['daily'].values, objective_shapes['daily'])
    weekly = tf.reshape(row['weekly'].values, objective_shapes['weekly'])
    target = tf.reshape(row['target'].values, objective_shapes['target'])
    oh_wd = tf.reshape(row['oh_wd'].values, objective_shapes['oh_wd'])
    oh_dh = tf.reshape(row['oh_dh'].values, objective_shapes['oh_dh'])
    timestamp = tf.reshape(row['timestamp'].values, objective_shapes['timestamp'])

    # important: this is a different parse function from the one in training
    # it returns target as a feature to use target values and timestamps for plotting
    return {'hourly': hourly,
            'daily': daily,
            'weekly': weekly,
            'target': target,
            'oh_wd': oh_wd,
            'oh_dh': oh_dh,
            'timestamp': timestamp}

In [169]:
dataset = test_dataset.map(lambda row: _parse_dataset_function(row))
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [170]:
timestamps_list = []
targets_list = []
with tf.compat.v1.Session() as sess:
    try:
        while True:
            item = sess.run(next_element)
            # get the timestamp, then the scalar value, then the string value, then convert it to datetime
            timestamps_list.append(to_datetime(item['timestamp'][0][0].decode()))
            # get the target, then the scalar value
            targets_list.append(item['target'][0][0])
    except:
        pass

In [171]:
len(timestamps_list), len(targets_list)

(0, 0)